In [1]:
import pandas as pd
import glob
import os
from pyspark.sql import SparkSession

In [ ]:
# download data(https://github.com/erkansirin78/datasets/raw/master/sensors_instrumented_in_an_office_building_dataset.zip) and unzip

In [ ]:
# unzipped files loaciton
files = glob.glob("/opt/spark/examples/datasets/KETI/*/*.csv")

# get single file from many files.
dataframes = []

for file in files:
    df = pd.read_csv(file, header=None)
    df['room'] = file.split("/opt/spark/examples/datasets/KETI/")[1].split(".csv")[0].split("/")[0]
    df['sensor'] = file.split("/opt/spark/examples/datasets/KETI/")[1].split(".csv")[0].split("/")[1]
    dataframes.append(df)

combined_data = pd.concat(dataframes, ignore_index=True)

combined_data.to_csv("concat_keti.csv", index=False, header=None)

In [2]:
#start spark session
spark = SparkSession.builder \
.appName("HomeWorkSparkDataClean") \
.master("local[2]") \
.config("spark.jars.packages","org.apache.spark:spark-avro_2.12:3.4.1") \
.getOrCreate()

23/11/17 13:14:24 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
23/11/17 13:14:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/train/miniconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b567df19-824b-4524-a21d-270b5ba0e381;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
:: resolution report :: resolve 178ms :: artifacts dl 6ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.4.1 from central in [default]
	org.tukaani#xz;1.9 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------


In [3]:
from pyspark.sql.types import *

custom_schema = StructType([
        StructField("timestamp",IntegerType(),True),
        StructField("value",FloatType(),True),
        StructField("room",StringType(),True),
        StructField("sensor",StringType(),True)
    ])

In [4]:
df = spark.read \
.format("csv") \
.option("header", False) \
.schema(custom_schema) \
.load("file:///home/train/00_spark_kafka_postgres_minio_docker_compose/spark/examples/final/concat_keti.csv")

In [5]:
df.limit(5).toPandas()

,timestamp,value,room,sensor
0,1377292829,24.219999,776,temperature
1,1377292834,24.240000,776,temperature
2,1377292839,24.240000,776,temperature
3,1377292844,24.240000,776,temperature
4,1377292849,24.240000,776,temperature


In [6]:
df.createOrReplaceTempView("keti")

In [23]:
df_1 = spark.sql("""

    SELECT DATE_TRUNC('minute', cast(timestamp as timestamp)) as time, cast(value as decimal(10,2)), room, sensor
    FROM keti
    
""")

df_1.printSchema()

df_1.limit(5).toPandas()

root
 |-- time: timestamp (nullable = true)
 |-- value: decimal(10,2) (nullable = true)
 |-- room: string (nullable = true)
 |-- sensor: string (nullable = true)



,time,value,room,sensor
0,2013-08-24 00:20:00,24.22,776,temperature
1,2013-08-24 00:20:00,24.24,776,temperature
2,2013-08-24 00:20:00,24.24,776,temperature
3,2013-08-24 00:20:00,24.24,776,temperature
4,2013-08-24 00:20:00,24.24,776,temperature


In [24]:
df_1.createOrReplaceTempView("keti_1")

In [25]:
df_temperature = spark.sql("""

    SELECT time, room, avg(value) as value
    FROM keti_1
    where sensor = 'temperature'
    group by time, room
    
""")

df_temperature.printSchema()

df_temperature.limit(5).toPandas()

root
 |-- time: timestamp (nullable = true)
 |-- room: string (nullable = true)
 |-- value: decimal(14,6) (nullable = true)



,time,room,value
0,2013-08-24 04:03:00,776,24.517500
1,2013-08-24 06:41:00,776,23.858333
2,2013-08-24 09:18:00,776,23.247500
3,2013-08-24 09:28:00,776,23.208333
4,2013-08-24 11:43:00,776,22.865000


In [26]:
df_temperature.createOrReplaceTempView("temperature")

In [27]:
df_pir = spark.sql("""

    SELECT time, room, avg(value) as value
    FROM keti_1
    where sensor = 'pir'
    group by time, room
    
""")

df_pir.printSchema()

df_pir.limit(5).toPandas()

root
 |-- time: timestamp (nullable = true)
 |-- room: string (nullable = true)
 |-- value: decimal(14,6) (nullable = true)



,time,room,value
0,2013-08-24 04:03:00,776,0.000000
1,2013-08-24 06:41:00,776,0.000000
2,2013-08-24 09:18:00,776,0.000000
3,2013-08-24 09:28:00,776,0.000000
4,2013-08-24 11:43:00,776,0.000000


In [28]:
df_pir.createOrReplaceTempView("pir")

In [29]:
df_light = spark.sql("""

    SELECT time, room, avg(value) as value
    FROM keti_1
    where sensor = 'light'
    group by time, room
    
""")

df_light.printSchema()

df_light.limit(5).toPandas()

root
 |-- time: timestamp (nullable = true)
 |-- room: string (nullable = true)
 |-- value: decimal(14,6) (nullable = true)



,time,room,value
0,2013-08-24 04:03:00,776,59.250000
1,2013-08-24 06:41:00,776,4.333333
2,2013-08-24 09:18:00,776,4.166667
3,2013-08-24 09:28:00,776,4.333333
4,2013-08-24 11:43:00,776,4.000000


In [30]:
df_light.createOrReplaceTempView("light")

In [31]:
df_humidity = spark.sql("""

    SELECT time, room, avg(value) as value
    FROM keti_1
    where sensor = 'humidity'
    group by time, room
    
""")

df_humidity.printSchema()

df_humidity.limit(5).toPandas()

root
 |-- time: timestamp (nullable = true)
 |-- room: string (nullable = true)
 |-- value: decimal(14,6) (nullable = true)



,time,room,value
0,2013-08-24 04:03:00,776,48.670000
1,2013-08-24 06:41:00,776,49.936667
2,2013-08-24 09:18:00,776,51.367500
3,2013-08-24 09:28:00,776,51.552500
4,2013-08-24 11:43:00,776,52.625000


In [32]:
df_humidity.createOrReplaceTempView("humidity")

In [33]:
df_co2 = spark.sql("""

    SELECT time, room, avg(value) as value
    FROM keti_1
    where sensor = 'co2'
    group by time, room
    
""")

df_co2.printSchema()

df_co2.limit(5).toPandas()

root
 |-- time: timestamp (nullable = true)
 |-- room: string (nullable = true)
 |-- value: decimal(14,6) (nullable = true)



,time,room,value
0,2013-08-24 04:03:00,776,591.500000
1,2013-08-24 06:41:00,776,517.916667
2,2013-08-24 09:18:00,776,497.500000
3,2013-08-24 09:28:00,776,492.916667
4,2013-08-24 11:43:00,776,486.500000


In [34]:
df_co2.createOrReplaceTempView("co2")

In [35]:
df_all = spark.sql("""

    SELECT t.room, t.time, t.value as temperature, p.value as pir, l.value as light, h.value as humidity, c.value as co2
    from temperature t
    inner join pir p on p.time = t.time and p.room = t.room
    inner join light l on l.time = t.time and l.room = t.room
    inner join humidity h on h.time = t.time and h.room = t.room
    inner join co2 c on c.time = t.time and c.room = t.room
    
""")

df_all.printSchema()

df_all.limit(5).toPandas()

root
 |-- room: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- temperature: decimal(14,6) (nullable = true)
 |-- pir: decimal(14,6) (nullable = true)
 |-- light: decimal(14,6) (nullable = true)
 |-- humidity: decimal(14,6) (nullable = true)
 |-- co2: decimal(14,6) (nullable = true)



,room,time,temperature,pir,light,humidity,co2
0,511,2013-08-24 02:04:00,22.620000,0.000000,250.500000,52.750000,389.000000
1,621A,2013-08-24 02:04:00,22.210000,0.000000,125.500000,55.230000,385.000000
2,644,2013-08-24 02:04:00,22.805000,0.000000,165.000000,52.385000,468.500000
3,648,2013-08-24 02:04:00,23.320000,0.000000,190.000000,50.320000,174.500000
4,656A,2013-08-24 02:04:00,24.370000,28.500000,176.500000,49.900000,578.500000


In [36]:
df_all.createOrReplaceTempView("keti_all")

In [40]:
df_2 = spark.sql("""

    SELECT cast(time as timestamp) as time,
    room,
    cast(cast(temperature as decimal(5,2)) as float) as temperature,
    cast(cast(pir as decimal(5,2)) as float) as pir,
    cast(cast(light as decimal(5,2)) as float) as light,
    cast(cast(humidity as decimal(5,2)) as float) as humidity,
    cast(cast(co2 as decimal(5,2)) as float) as co2
    from keti_all
    order by time, room
    
""")

df_2.printSchema()

df_2.limit(5).toPandas()

root
 |-- time: timestamp (nullable = true)
 |-- room: string (nullable = true)
 |-- temperature: float (nullable = true)
 |-- pir: float (nullable = true)
 |-- light: float (nullable = true)
 |-- humidity: float (nullable = true)
 |-- co2: float (nullable = true)



,time,room,temperature,pir,light,humidity,co2
0,2013-08-24 02:04:00,510,23.459999,0.0,204.0,52.619999,404.0
1,2013-08-24 02:04:00,511,22.620001,0.0,250.5,52.750000,389.0
2,2013-08-24 02:04:00,621,25.590000,14.0,67.5,49.119999,500.0
3,2013-08-24 02:04:00,621A,22.209999,0.0,125.5,55.230000,385.0
4,2013-08-24 02:04:00,640,22.740000,0.0,8.0,52.480000,455.0


In [41]:
df_2.count()

533696

In [42]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import  *
import time

In [43]:
# export cleaned and transformed data parguet format.

start_time = time.time()

df_2.coalesce(1).write \
.format("parquet") \
.mode("overwrite") \
.option("compression", "none") \
.save("file:///opt/spark/examples/datasets/concat_keti_parq")


print("----------- %s secs ----------" %(time.time() - start_time))

! ls -l /opt/spark/examples/datasets/

----------- 118.29868507385254 secs ----------
total 4
drwxr-xr-x.  2 train train  184 Nov 17 13:49 concat_keti_parq
drwxrwxr-x. 53 train train 4096 Oct 21 18:29 KETI


In [44]:
spark.stop()